In [1]:
import pandas as pd
import numpy as np

In [13]:
dividends = pd.read_csv('/Users/evelynli/Documents/Work/ASL/ETF Hedging Data/ETF/processed data/etf_dividends1.csv')
dividends.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
dividends['Date'] = pd.to_datetime(dividends['Date'])
dividends = dividends.set_index('Date')
dividends.fillna(0, inplace=True)
dividends

,BIL,IEF,TLT
Date,,,
2008-02-01,0.230,0.281,0.329
2008-03-03,0.174,0.280,0.348
2008-04-01,0.140,0.311,0.406
2008-05-01,0.108,0.280,0.336
2008-06-02,0.094,0.307,0.327
...,...,...,...
2024-04-01,0.405,0.276,0.312
2024-05-01,0.388,0.271,0.308
2024-06-03,0.405,0.276,0.308


In [14]:
nav = pd.read_csv('/Users/evelynli/Documents/Work/ASL/Historical Data/ETF prices.csv', index_col = 'Date', parse_dates = True)
nav.sort_index(inplace=True)
nav

,BIL,IEF,TLT
Date,,,
2008-12-31,91.7200,98.3700,119.0300
2009-01-02,91.7000,97.1800,116.6100
2009-01-05,91.7000,96.8400,112.5400
2009-01-06,91.7000,96.7900,111.8500
2009-01-07,91.7000,96.8900,111.8900
...,...,...,...
2024-07-26,91.7627,94.9687,92.9443
2024-07-29,91.7762,95.1595,93.4474
2024-07-30,91.7893,95.3563,93.7941


In [15]:
def calculate_daily_dividend_multiplier(nav, dividends):
    #nav = nav.sort_index(ascending=False)
    #dividends = dividends.sort_index(ascending=False)

    multiplier = pd.DataFrame(1, index=nav.index, columns=nav.columns)
    print(multiplier)

    for etf in nav.columns:
        for i in range(1, len(nav)):
            current_date = nav.index[i]
            previous_date = nav.index[i - 1]
            
            if previous_date in dividends.index and dividends.loc[previous_date, etf] != 0:
                dividend = dividends.loc[previous_date, etf]
                nav_t = nav.loc[current_date, etf]
                multiplier.loc[current_date, etf] = multiplier.loc[previous_date, etf] * (1 - dividend / nav_t)
            else:
                multiplier.loc[current_date, etf] = multiplier.loc[previous_date, etf]

    multiplier = multiplier.sort_index(ascending=True)

    return multiplier


dividend_multiplier = calculate_daily_dividend_multiplier(nav, dividends)

dividend_multiplier


            BIL  IEF  TLT
Date                     
2008-12-31    1    1    1
2009-01-02    1    1    1
2009-01-05    1    1    1
2009-01-06    1    1    1
2009-01-07    1    1    1
...         ...  ...  ...
2024-07-26    1    1    1
2024-07-29    1    1    1
2024-07-30    1    1    1
2024-07-31    1    1    1
2024-08-01    1    1    1

[3923 rows x 3 columns]


/var/folders/kk/4hst7jm10nj04j3404bsj2yr0000gn/T/ipykernel_2758/556583666.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.999825479930192' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  multiplier.loc[current_date, etf] = multiplier.loc[previous_date, etf] * (1 - dividend / nav_t)
/var/folders/kk/4hst7jm10nj04j3404bsj2yr0000gn/T/ipykernel_2758/556583666.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9971822033898305' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  multiplier.loc[current_date, etf] = multiplier.loc[previous_date, etf] * (1 - dividend / nav_t)
/var/folders/kk/4hst7jm10nj04j3404bsj2yr0000gn/T/ipykernel_2758/556583666.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error

,BIL,IEF,TLT
Date,,,
2008-12-31,1.000000,1.000000,1.000000
2009-01-02,1.000000,1.000000,1.000000
2009-01-05,1.000000,1.000000,1.000000
2009-01-06,1.000000,1.000000,1.000000
2009-01-07,1.000000,1.000000,1.000000
...,...,...,...
2024-07-26,0.871758,0.714123,0.648086
2024-07-29,0.871758,0.714123,0.648086
2024-07-30,0.871758,0.714123,0.648086


In [16]:
adjusted_nav = pd.DataFrame(index=nav.index, columns=nav.columns)
for etf in nav.columns:
    adjusted_nav[etf] = nav[etf] * dividend_multiplier[etf]

adjusted_nav.to_csv('/Users/evelynli/Documents/Work/ASL/ETF Hedging Data/ETF/processed data/adjusted_nav1.csv')
adjusted_nav


,BIL,IEF,TLT
Date,,,
2008-12-31,91.720000,98.370000,119.030000
2009-01-02,91.700000,97.180000,116.610000
2009-01-05,91.700000,96.840000,112.540000
2009-01-06,91.700000,96.790000,111.850000
2009-01-07,91.700000,96.890000,111.890000
...,...,...,...
2024-07-26,79.994828,67.819294,60.235899
2024-07-29,80.006597,67.955548,60.561951
2024-07-30,80.018017,68.096087,60.786642


In [11]:
import os

directory = "/Users/evelynli/Documents/Work/ASL/Duration Data"

duration = pd.DataFrame(index= adjusted_nav.index)

for filename in os.listdir(directory):
    if filename.endswith(".xlsx") and "duration history" in filename:
        filepath = os.path.join(directory, filename)
        df = pd.read_excel(filepath, index_col='Date')
    
        etf_name = filename.split(' ')[0]  # Assuming the ETF name is before the first space
        df.columns = [etf_name]
        df = df.reindex(adjusted_nav.index).ffill()
        duration[etf_name] = df[etf_name]

duration

,SGOV,USFR,BIL,IEF,TLT
Date,,,,,
2021-01-04,0.1027,1.4361,0.0602,7.8522,19.4815
2021-01-05,0.0989,1.4706,0.0579,7.8904,19.3257
2021-01-06,0.0963,1.4871,0.0567,7.7793,18.9042
2021-01-07,0.0881,1.4573,0.1302,7.8626,19.0954
2021-01-08,0.0849,1.4544,0.1274,7.8442,18.8167
...,...,...,...,...,...
2024-05-26,0.0782,1.3898,0.0770,7.1361,16.4228
2024-05-27,0.0782,1.3898,0.0770,7.1361,16.4228
2024-05-28,0.2189,1.3925,0.0769,7.1321,16.3429


In [12]:
dv01 = pd.DataFrame(index=adjusted_nav.index, columns=duration.columns)

for etf in dv01.columns:
    dv01[etf] = adjusted_nav[etf] * duration[etf]*0.0001

dv01.to_csv('/Users/evelynli/Documents/Work/ASL/ETF Hedging Data/ETF/processed data/ETF_DV01.csv')

In [13]:
dv01

,SGOV,USFR,BIL,IEF,TLT
Date,,,,,
2021-01-04,0.000947,0.003304,0.000509,0.088079,0.281909
2021-01-05,0.000912,0.003384,0.000489,0.088264,0.276892
2021-01-06,0.000888,0.003422,0.000479,0.086458,0.264888
2021-01-07,0.000813,0.003353,0.001100,0.087185,0.266372
2021-01-08,0.000783,0.003347,0.001077,0.086738,0.261554
...,...,...,...,...,...
2024-05-26,0.000787,0.006989,0.000706,0.066220,0.149927
2024-05-27,0.000787,0.006989,0.000706,0.066220,0.149927
2024-05-28,0.002204,0.007007,0.000706,0.065856,0.147149
